In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd


In [2]:
df = pd.read_csv('nepse_data.csv')

In [3]:
df = df.sort_values('Date')

In [4]:
#Conversion line
hi_val = df['High'].rolling(window=9).max()
low_val = df['Low'].rolling(window=9).min()
df['Conversion'] = (hi_val + low_val) /2

In [5]:
# Base line
hi_val2 = df['High'].rolling(window=26).max()
low_val2 = df['Low'].rolling(window=26).min()
df['Baseline'] = (hi_val2 + low_val2) /2

In [6]:
#leading span A
df['SpanA'] = (df['Conversion']+ df['Baseline'])/2

In [7]:
#leading span B
hi_val3 = df['High'].rolling(window=52).max()
low_val3 = df['Low'].rolling(window=52).min()
df['SpanB'] = (hi_val3 + low_val3) /2

In [8]:
#lagging span
df['Lagging'] = df['Close'].shift(-26)

In [9]:
#create Candlestick
candle = go.Candlestick(x=df.index, open=df['Open'].shift(-26),high=df['High'].shift(-26),low=df['Low'].shift(-26),close=df['Close'].shift(-26), name='Candlestick')

In [10]:
def get_fill_color(label):
    if label>=1:
        return 'rgba(0,250,0,0.4)'
    else:
       return 'rgba(250,0,0,0.4)' 

In [12]:
df1 = df

fig = go.Figure()
fig = make_subplots(rows=2,cols=1,shared_xaxes="columns", shared_yaxes="rows", column_width=[1], row_heights=[0.6,0.1], subplot_titles=["Candlestick", "Volume"])

df['label'] = np.where(df['SpanA'] > df['SpanB'], 1,0)
df['group'] = df['label'].ne(df['label'].shift()).cumsum()

df = df.groupby('group')

dfs = []
for , data in df:
    dfs.append(data)

for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y= df.SpanA, line=dict(color='rgba(0,0,0,0)'), name ='Cloud'))
    fig.add_traces(go.Scatter(x=df.index, y= df.SpanB, line=dict(color='rgba(0,0,0,0)'), fill='tonexty', fillcolor=get_fill_color(df['label'].iloc[0]), name='Cloud'))

baseline = go.Scatter(x=df1.index, y=df1['Baseline'].shift(-26), line=dict(color='red',width = 1),name="Baseline")
conversion = go.Scatter(x=df1.index, y=df1['Conversion'].shift(-26), line=dict(color='blue',width = 1),name="Conversion")
lagging = go.Scatter(x=df1.index, y=df1['Lagging'].shift(-26), line=dict(color='green',width = 1),name="Lagging")
span_a = go.Scatter(x=df1.index, y=df1['SpanA'].shift(0), line=dict(color='green',width = 1),name="SpanA")
span_b = go.Scatter(x=df1.index, y=df1['SpanB'].shift(0), line=dict(color='red',width = 1),name="SpanB")

fig.add_trace(candle)
fig.add_trace(baseline)
fig.add_trace(conversion)
fig.add_trace(lagging)
fig.add_trace(span_a)
fig.add_trace(span_b)

fig.add_trace(go.Bar(x=df1.index, y=df1['Volume'], name='Volume'),col=1,row=2)
fig.update_layout(width=1200,height=800,xaxis_rangeslider_visible=False,showlegend=True)

fig.show()



TypeError: tuple indices must be integers or slices, not str